In [16]:
import pandas as pd
import dvc.api
import mlflow
import matplotlib.pyplot as plt

import seaborn as sns

In [71]:
path = 'data\AdSmartABdata.csv'
repo = 'https://github.com/SameC137/abtest-mlops'
rev = 'v2'
data_url = dvc.api.get_url(path=path, repo=repo,rev=rev)

collected_data = pd.read_csv(data_url,index_col=0)
collected_data

,auction_id,experiment,date,hour,device_make,platform_os,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,0,0
...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,0,0


## Select only users with a response

In [73]:
responded=collected_data.loc[(collected_data["yes"]==1) | (collected_data["no"]==1)]



# Remove auction Id

In [74]:
features= responded.drop(["auction_id","no"],axis=1, inplace=False)
features=features.reset_index(drop=True)

In [75]:
y=features["yes"]

In [51]:


# lb = LabelEncoder() 

# features["experiment"]=lb.fit_transform(features["experiment"])

# features["device_make"]=lb.fit_transform(features["device_make"])

# features["browser"]=lb.fit_transform(features["browser"])

# features["date"]=lb.fit_transform(features["date"])

# features.drop("yes",axis=1,inplace=True)

In [79]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


def encode_scale_features(df,columns):
    lb=LabelEncoder()
    norm = MinMaxScaler()
    for i in columns:
        df[i]=lb.fit_transform(df[i])   
    norm_fit = norm.fit_transform(df)
    out=pd.DataFrame(norm_fit,columns=df.columns)
    return out

In [82]:
if rev=="v2":
    feat=["experiment","device_make","date"]
elif rev=="v3":
    feat=["experiment","device_make","browser","date"]
else:
    feat=["experiment","device_make","browser","plaform_os","date"]

features=encode_scale_features(features,feat)
    

In [52]:


# creating scaler scale var.
# norm = MinMaxScaler()
# fit the scal
# norm_fit = norm.fit_transform(features)

# X=pd.DataFrame(norm_fit,columns=features.columns)

In [83]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.3, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.33, random_state=1)


In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import mean_squared_error, accuracy_score,log_loss
# import mlflow
# import mlflow.sklearn


# from mlflow.models.signature import infer_signature
# from mlflow.utils.environment import _mlflow_conda_env


# model = LogisticRegression()
# with mlflow.start_run() as run:
#     model.fit(X_train y_train)
#     pred = cls.predict(X_test)
#     mlflow.log_metric(f"accuracy", kfold_scores.mean())
#     mlflow.log_metric(f"std_accuracy", kfold_scores.std())
#     print(mean_squared_error(y_test, pred))
#     print("Logged data and model in run {}".format(run.info.run_id))
    

In [ ]:
# def eval_metrics(actual, pred):
#     rmse = np.sqrt(mean_squared_error(actual, pred))
#     mae = mean_absolute_error(actual, pred)
#     r2 = r2_score(actual, pred)
#     return rmse, mae, r2
    

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, log_loss
import mlflow
import mlflow.sklearn

from urllib.parse import urlparse

from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import numpy as np


with mlflow.start_run(run_name='untuned_linear_regression'):
    
    mlflow.log_param('data_version', rev)
    feature_cols=pd.DataFrame(list(X_train.columns))
    feature_cols.to_csv('features.csv',header=False,index=False)
    mlflow.log_artifact("features.csv")

    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    loss=log_loss(y_val,pred)
    acc = accuracy_score(y_val, pred)

    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("log_loss", loss)
    
    mlflow.log_metric("accuracy", acc)
    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(model, "model", registered_model_name="LinearRegressionModel")
    else:
        mlflow.sklearn.log_model(model, "model")






In [67]:

feature_importances = pd.DataFrame((model.coef_).transpose() , index=X_train.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)
# X_train.columns.tolist()

,importance
device_make,0.364569
experiment,0.154695
hour,-0.003770
browser,-0.062687
date,-0.063641


In [84]:
from xgboost import XGBClassifier


with mlflow.start_run(run_name='untuned_kgboost'):
    
    mlflow.log_param('data_version', rev)
    feature_cols=pd.DataFrame(list(X_train.columns))
    feature_cols.to_csv('features.csv',header=False,index=False)
    mlflow.log_artifact("features.csv")

    model = XGBClassifier()
    model.fit(X_train, y_train)
    
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    loss=log_loss(y_val,pred)
    acc = accuracy_score(y_val, pred)

    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("log_loss", loss)
    
    mlflow.log_metric("accuracy", acc)
    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(model, "model", registered_model_name="XGBoost")
    else:
        mlflow.sklearn.log_model(model, "model")






D:\Users\same\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:54:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [89]:

from sklearn.tree import DecisionTreeClassifier


with mlflow.start_run(run_name='untuned_decisiontree'):
    
    mlflow.log_param('data_version', rev)
    feature_cols=pd.DataFrame(list(X_train.columns))
    feature_cols.to_csv('features.csv',header=False,index=False)
    mlflow.log_artifact("features.csv")

    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    loss=log_loss(y_val,pred)
    acc = accuracy_score(y_val, pred)

    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("log_loss", loss)
    
    mlflow.log_metric("accuracy", acc)
    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(model, "model", registered_model_name="DecisionTreeClassifier")
    else:
        mlflow.sklearn.log_model(model, "model")

In [87]:
feature_importances = pd.DataFrame((model.feature_importances_).transpose() , index=X_train.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)
# X_train.columns.tolist()

,importance
hour,0.345331
device_make,0.237050
date,0.211202
experiment,0.106343
browser,0.100074


In [88]:
class CreateModel:
    def __init__(self, X_train, X_test, y_train, y_test,data_version,name,model):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        self.model=NULL
        self.featureImportance=[]
        self.name=name
        self.data_version=data_version
    def train(self,params):
        with mlflow.start_run(run_name=self.name):
            mlflow.log_param('data_version', this.data_version)
            feature_cols=pd.DataFrame(list(self.X_train.columns))
            feature_cols.to_csv('features.csv',header=False,index=False)
            mlflow.log_artifact("features.csv")

            model = self.model(**params)
            model.fit(self.X_train, self.y_train)

            pred = model.predict(self.X_test)
            rmse = np.sqrt(mean_squared_error(self.y_test, pred))
            loss=log_loss(self.y_test,pred)
            acc = accuracy_score(self.y_test, pred)


            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("log_loss", loss)

            mlflow.log_metric("accuracy", acc)

            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name=self.name)
            else:
                mlflow.sklearn.log_model(model, "model")
    def trainKFold(self,folds,params):
        with mlflow.start_run(run_name=self.name):
            mlflow.log_param('data_version', this.data_version)
            feature_cols=pd.DataFrame(list(self.X_train.columns))
            feature_cols.to_csv('features.csv',header=False,index=False)
            mlflow.log_artifact("features.csv")
            
            kf=KFold(n_splits=folds, random_state=None)
            model = self.model(**params)
            model.fit(self.X_train, self.y_train)

            pred = model.predict(self.X_test)
            rmse = np.sqrt(mean_squared_error(self.y_test, pred))
            loss=log_loss(self.y_test,pred)
            acc = accuracy_score(self.y_test, pred)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("log_loss", loss)

            mlflow.log_metric("accuracy", acc)

            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name=self.name)
            else:
                mlflow.sklearn.log_model(model, "model")
    
    def getFeatureImportance(self):
        feature_importances = pd.DataFrame((self.model.feature_importances_).transpose() , index=self.X_train.columns.tolist(), columns=['importance'])
        return feature_importances.sort_values('importance', ascending=False)

SyntaxError: invalid syntax (<ipython-input-88-e5323cbb8e77>, line 1)